# **Sim Output to EcoCyc Data Comparison**

In [ ]:
import numpy as np
import pandas as pd
import statistics
import math
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
logTransposeConst = 0.00000000000000001


def zeroFlux(key, dictionary):
    fluxes = dictionary[key]
    for num in fluxes:
        if not num == 0:
            return False
    return True


def createTuples(dictionary):
    result = []
    keys = dictionary.keys()
    for key in keys:
        mean = statistics.mean(dictionary[key])
        result.append(tuple((key, mean)))
    return result


def getZeroFluxReactions(reactionDictionary):
    reactions = reactionDictionary.keys()
    zeroFluxReactions = []

    for key in reactions:
        if zeroFlux(key, reactionDictionary):
            zeroFluxReactions.append(key)
            
    return zeroFluxReactions


def createHeatMapFluxes(reactionDictionary, mapHeight, annot_grid):
    reactions = reactionDictionary.keys()
    zeroFluxReactions = []
    nonZeroFlux = {}

    for key in reactions:
        if zeroFlux(key, reactionDictionary):
            zeroFluxReactions.append(key)
        else:
            res = [abs(ele) for ele in reactionDictionary[key]]
            nonZeroFlux[key] = res

    transformedData = {}
    for key in nonZeroFlux.keys():
        new = []
        old = nonZeroFlux[key]
        for val in old:
            new.append(math.log(val + logTransposeConst))
        transformedData[key] = new

    tuples = createTuples(transformedData)
    tuples.sort(key=lambda x: x[1])

    sortedReactions = {}
    for elem in tuples:
        key = elem[0]
        sortedReactions[key] = transformedData[key]

    df = pd.DataFrame.from_dict(sortedReactions, orient='index')
    df.index.name = 'Reactions'
    df.reset_index(inplace=False)

    fig, ax = plt.subplots(figsize=(50, mapHeight))
    plt.xlabel("Seconds")
    if annot_grid:
        sns.heatmap(df, cmap="rocket_r", linewidths=.5, annot=True)
    else:
        sns.heatmap(df, cmap="rocket_r")
        
    return 


def notPresent(key, complexes):
    amounts = complexes[key]
    for num in amounts:
        if not num == 0:
            return False
    return True


def createHeatMapComplexes(complexes, mapHeight, annot_grid):
    complexNames = complexes.keys()
    zeroQuantComplexes = []
    presentComplexes = {}

    for key in complexes:
        if notPresent(key, complexes):
            zeroQuantComplexes.append(key)
        else:
            presentComplexes[key] = complexes[key]

    transformedData = {}
    for key in presentComplexes.keys():
        new = []
        old = presentComplexes[key]
        for val in old:
            new.append(math.log(val + logTransposeConst))
        transformedData[key] = new

    tuples = createTuples(transformedData)
    tuples.sort(key=lambda x: x[1])

    sortedComplexes = {}
    for elem in tuples:
        key = elem[0]
        sortedComplexes[key] = transformedData[key]

    df = pd.DataFrame.from_dict(sortedComplexes, orient='index')
    df.index.name = 'Reactions'
    df.reset_index(inplace=False)

    fig, ax = plt.subplots(figsize=(50, mapHeight))
    plt.xlabel("Seconds")
    if annot_grid:
        hmap = sns.heatmap(df, cmap="rocket_r", linewidths=.5, annot=True)
    else:
        hmap = sns.heatmap(df, cmap="rocket_r")

    result = [hmap, zeroQuantComplexes]

    return result

In [ ]:
simData = np.load(r"../../out/geneRxnVerifData/output.npy", allow_pickle=True, encoding='ASCII')
fluxesWithCaption = simData.tolist()['agents']['0']['listeners']['fba_results']['estimated_fluxes']
complexes = simData.tolist()['agents']['0']['bulk']

ecData = pd.read_csv(r"../new genes/geneData.txt")
ecReactions = ecData["reactionID"].values.tolist()
temp = []
for word in ecReactions:
    if not word == "No Reaction Found":
        temp.append(word)
ecReactions = temp

metabolData = np.load(r"../../out/stoichiometry.npy", allow_pickle=True, encoding='ASCII')
rxn_metabolites = metabolData.tolist()

In [ ]:
rxn_metabolites["ISOCIT-CLEAV-RXN"]

### **Sim Output: Reaction Flux Analysis**

#### *Run Following Block to visualize reactions with Flux throughout time frame*

In [ ]:
createHeatMapFluxes(fluxesWithCaption, 650, True)

#### *Run Following Block to identify reactions with Zero Flux throughout time frame*

In [ ]:
rxns = getZeroFluxReactions(fluxesWithCaption)
print(len(rxns), "zero flux reactions found.")
enter = input("Press 'Enter' to show reaction names. Click any other key to contintue")
if enter == "":
    print(rxns)

### **Sim Output: Complex Analysis**

#### *Run Following Block to visualize complexes present during time frame*

In [ ]:
createHeatMapComplexes(complexes, 650, False)

#### *Run Following Block to identify complexes not present throughout time frame*

In [ ]:
zeroQuant = []
for key in complexes:
    if notPresent(key, complexes):
        zeroQuant.append(key)
        
print(len(zeroQuant), "complexes not found.")
enter = input("Press 'Enter' to show reaction names. Click any other key to contintue")
if enter == "":
    print(zeroQuant)     

#### *Run Following Block to identify complexes with incorrect compartments*

In [ ]:
incorrectCompartments = []
for complex_ in complexes:
    if not (complex_[len(complex_) - 1 - 2] == '[' and complex_[len(complex_) - 1] == ']'):
        incorrectCompartments.append(complex_)

print(len(incorrectCompartments), "incorrect compartments found.")
enter = input("Press 'Enter' to show complex names. Click any other key to contintue")
if enter == "":
    print(incorrectCompartments)

### **EcoCyc Comparison: Reaction Flux**

#### *Run following block before any other block in this section* **Block A**

In [ ]:
reactionsSim = fluxesWithCaption.keys()

foundReactions = {}
unfoundReactions = []

for reaction in ecReactions:
    x = 0
    for reactionSim in reactionsSim:
        if reaction in reactionSim:
            index = reactionSim.find(reaction)
            indexOfNextChar = index + len(reaction)
            if indexOfNextChar >= len(reactionSim) or not (reactionSim[indexOfNextChar].isdigit() or reactionSim[indexOfNextChar].isalpha()):
                x = 1
                key = "(" + reaction + ")" + reactionSim
                foundReactions[key] = fluxesWithCaption[reactionSim]
    if x == 0:
        unfoundReactions.append(reaction)

#### *Run following block to identify EcoCyc reactions not found in Simulation*

In [ ]:
print(len(unfoundReactions), "reactions not present.")
enter = input("Press 'Enter' to show complex names. Click any other key to contintue")
if enter == "":
    print(unfoundReactions)

#### *Run following block to identify EcoCyc reactions with No Flux in Simulation*

In [ ]:
rxns1 = getZeroFluxReactions(foundReactions)
print(len(rxns1), "zero flux reactions found.")
enter = input("Press 'Enter' to show reaction names. Click any other key to contintue")
if enter == "":
    print(rxns1)

#### *Run following block to visualize non-zero fluxes for EcoCyc reactions found in Simulation*

In [ ]:
createHeatMapFluxes(foundReactions, 25, True)

### **EcoCyc Comparison: Stoichiometry**

#### *Run following block to validate metabolite compartments*

In [ ]:
rxn_metabolites["ISOCIT-CLEAV-RXN"]

In [ ]:
incorrect = []

for reaction in rxn_metabolites.keys():
    for metabolite in rxn_metabolites[reaction].keys():
        if not (metabolite[len(metabolite) - 1 - 2] == '[' and metabolite[len(metabolite) - 1] == ']'):
            incorrect.append((reaction, metabolite))

print(len(incorrect), "incorrect compartments found.")
enter = input("Press 'Enter' to show metabolite names. Click any other key to contintue")
if enter == "":
    print(incorrect)

In [ ]:
rxn_metabolites["ISOCIT-CLEAV-RXN"]

#### *Run following block to validate complexation/stoichiometry for EcoCyc reactions found in Sim*

**Please run Block A first**

In [126]:
incorrect = []
ecDict = dict(zip(ecData.reactionID, ecData.stoichiometry))

for reaction in foundReactions.keys():
    x = 0
    origKey = ""
    newKey = ""
    
    for i in range(len(reaction)):
        if x >= 2:
            newKey = reaction[i:]
            break
        if reaction[i] == '(' or reaction[i] == ')':
            x += 1
        else:
            origKey += reaction[i]
    
    ec_stoich1 = ecDict[origKey]
    sim_stoich = rxn_metabolites[newKey]
    ec_stoich = dict(ec_stoich.split(":") for subString in str.split(","))
    
    temp = sim_stoich.keys()
    sim_stoich_new = {}
    
    for metabolite in temp:
        woutComp = metabolite[0:len(metabolite) - 3]
        if woutComp not in sim_stoich_new.keys():
            sim_stoich_new[woutComp] = sim_stoich[metabolite]
        else:
            temp = [sim_stoich_new[woutComp], sim_stoich[metabolite]]
            sim_stoich_new[woutComp] = temp
    
    errors = []
    
    for key in sim_stoich_new.keys():
        if key in ec_stoich.keys():
            a = set(sim_stoich[key])
            b = set(ec_stoich[key])
            if not a == b:
                    errors.append(("Discrepency in stoichiometry:", reaction, key, sim_stoich[key], ec_stoich[key]))            
        else:
            errors.append(("Metabolite not in EcoCyc Data:", key))

ValueError: dictionary update sequence element #0 has length 4; 2 is required